In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import pickle
# from statsmodels.graphics.gofplots import qqplot

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
import sklearn.gaussian_process as gp
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.dates as mdates
import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from skopt import gp_minimize
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from lightgbm import LGBMRegressor
import nbformat
from bayes_opt import BayesianOptimization
import xgboost as xgb
from statsmodels.graphics.gofplots import qqplot_2samples
import random

C:\Users\balag\AppData\Local\Temp\ipykernel_10920\2598249271.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Set random seed
np.random.seed(30)
random.seed(30)

In [3]:
data = pd.read_csv(r'D:\UTD\UTDFall2023\Calibration-of-LoRaNodes-using-Super-Learners\data\calibrate.csv')
data

,dateTime,pm10_grimm,pm2_5_grimm,pm1_grimm,inhalable_grimm,thoracic_grimm,alveolic_grimm,NH3_loRa,CO_loRa,NO2_loRa,...,Humidity_loRa,pm1Palas,pm2_5Palas,pm4Palas,pm10Palas,pmTotalPalas,dCnPalas,temperaturePalas,humidityPalas,pressurehPalas
0,04/08/2019 03:22:30 +0000,8.06,7.48,6.06,8.06,8.06,7.74,0.02,0,4.92,...,67,7.500000,9.210000,11.390000,13.516667,15.256667,191.830000,28.303333,43.425000,987.790000
1,04/08/2019 03:23:00 +0000,7.84,7.04,6.06,7.84,7.82,7.32,0.02,0,4.90,...,68,7.493333,9.210000,11.386667,13.506667,15.246667,191.786667,28.286667,43.316667,987.850000
2,04/08/2019 03:23:30 +0000,8.98,7.58,6.08,8.98,8.96,8.22,0.02,0,4.90,...,68,7.490000,9.206667,11.380000,13.493333,15.226667,191.736667,28.250000,43.301667,987.810000
3,04/08/2019 03:24:00 +0000,8.90,7.60,5.96,8.90,8.74,7.80,0.02,0,4.90,...,68,7.490000,9.200000,11.370000,13.480000,15.206667,191.703333,28.215000,43.483333,987.810000
4,04/08/2019 03:24:30 +0000,9.72,7.72,6.14,9.72,9.66,8.40,0.02,0,4.90,...,68,7.490000,9.200000,11.360000,13.466667,15.193333,191.676667,28.205000,43.381667,987.810000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,04/10/2019 10:52:30 +0000,8.64,5.62,4.94,8.72,8.18,6.08,0.14,0,4.71,...,52,5.913333,6.883333,7.923333,12.833333,20.376667,204.200000,29.251667,30.115000,980.160000
2059,04/10/2019 10:53:00 +0000,9.72,5.98,5.20,33.96,11.90,6.86,0.14,0,4.73,...,52,5.923333,6.853333,7.813333,12.610000,20.093333,205.363333,29.228333,30.173333,980.280000
2060,04/10/2019 10:54:00 +0000,11.14,6.82,5.10,30.24,13.58,7.72,0.14,0,4.76,...,52,5.986667,6.866667,7.680000,12.440000,19.880000,208.336667,29.238333,30.260000,980.038333
2061,04/10/2019 10:54:30 +0000,10.02,6.66,5.12,33.36,14.00,7.46,0.14,0,4.73,...,52,6.023333,6.903333,7.716667,12.476667,19.916667,209.700000,29.180000,30.353333,980.218333


In [4]:
data.columns

Index(['dateTime', 'pm10_grimm', 'pm2_5_grimm', 'pm1_grimm', 'inhalable_grimm',
       'thoracic_grimm', 'alveolic_grimm', 'NH3_loRa', 'CO_loRa', 'NO2_loRa',
       'C3H8_loRa', 'C4H10_loRa', 'CH4_loRa', 'H2_loRa', 'C2H5OH_loRa',
       'P1_lpo_loRa', 'P1_ratio_loRa', 'P1_conc_loRa', 'P2_lpo_loRa',
       'P2_ratio_loRa', 'P2_conc_loRa', 'Temperature_loRa', 'Pressure_loRa',
       'Humidity_loRa', 'pm1Palas', 'pm2_5Palas', 'pm4Palas', 'pm10Palas',
       'pmTotalPalas', 'dCnPalas', 'temperaturePalas', 'humidityPalas',
       'pressurehPalas'],
      dtype='object')

In [5]:
df= pd.concat([data.iloc[:,6], data.iloc[:,0:6],data.iloc[:,15:data.shape[1]]],axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2063 entries, 0 to 2062
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   alveolic_grimm    2063 non-null   float64
 1   dateTime          2063 non-null   object 
 2   pm10_grimm        2063 non-null   float64
 3   pm2_5_grimm       2063 non-null   float64
 4   pm1_grimm         2063 non-null   float64
 5   inhalable_grimm   2063 non-null   float64
 6   thoracic_grimm    2063 non-null   float64
 7   P1_lpo_loRa       2063 non-null   int64  
 8   P1_ratio_loRa     2063 non-null   float64
 9   P1_conc_loRa      2063 non-null   float64
 10  P2_lpo_loRa       2063 non-null   int64  
 11  P2_ratio_loRa     2063 non-null   float64
 12  P2_conc_loRa      2063 non-null   float64
 13  Temperature_loRa  2063 non-null   float64
 14  Pressure_loRa     2063 non-null   int64  
 15  Humidity_loRa     2063 non-null   int64  
 16  pm1Palas          2063 non-null   float64


In [6]:
def data_checker(df):
    # data_frame = df.iloc[:,1:16]
    data_frame =  df.iloc[:,7:-3] 
    # Palas limits are 0-100 mg/m3
    #BME limits are :Temp -40C to 85C
                   #:Pressure 300hPa to 1100 hPa or  300*100Pa to 1100*100 Pa
                   #:Humidity 0% to 100%
     #PPD42NS :Operating Temp is 0C to 45C
    idx = data_frame[(data_frame['Temperature_loRa']>=0) & (data_frame['Temperature_loRa']<=45) &
                    (data_frame['Pressure_loRa']>=300*100) & (data_frame['Pressure_loRa']<=1100*100) &
                    (data_frame['Humidity_loRa']>=0) & (data_frame['Humidity_loRa']<=100) &
                    (data_frame['pm1Palas']>=0) & (data_frame['pm2_5Palas']>=0) &
                    (data_frame['pm4Palas']>=0) & (data_frame['pm10Palas']>=0) &
                    (data_frame['pmTotalPalas']>=0) & (data_frame['dCnPalas']>=0)&
                    (data_frame['pm1Palas']<=100000) & (data_frame['pm2_5Palas']<=100000) &
                    (data_frame['pm4Palas']<=100000) & (data_frame['pm10Palas']<=100000) &
                    (data_frame['pmTotalPalas']<=100000) & (data_frame['dCnPalas']<=100000)].index
                    # &
                    # (data_frame['P1_conc_loRa'] + data_frame['P2_conc_loRa']>=0) &
                    # (data_frame['P1_conc_loRa'] + data_frame['P2_conc_loRa']<=28000)].index
    return df.loc[idx]
df = data_checker(df)
df

,alveolic_grimm,dateTime,pm10_grimm,pm2_5_grimm,pm1_grimm,inhalable_grimm,thoracic_grimm,P1_lpo_loRa,P1_ratio_loRa,P1_conc_loRa,...,Humidity_loRa,pm1Palas,pm2_5Palas,pm4Palas,pm10Palas,pmTotalPalas,dCnPalas,temperaturePalas,humidityPalas,pressurehPalas
0,7.74,04/08/2019 03:22:30 +0000,8.06,7.48,6.06,8.06,8.06,2556,0.02,9.48,...,67,7.500000,9.210000,11.390000,13.516667,15.256667,191.830000,28.303333,43.425000,987.790000
1,7.32,04/08/2019 03:23:00 +0000,7.84,7.04,6.06,7.84,7.82,0,0.00,0.62,...,68,7.493333,9.210000,11.386667,13.506667,15.246667,191.786667,28.286667,43.316667,987.850000
2,8.22,04/08/2019 03:23:30 +0000,8.98,7.58,6.08,8.98,8.96,86608,0.58,299.81,...,68,7.490000,9.206667,11.380000,13.493333,15.226667,191.736667,28.250000,43.301667,987.810000
3,7.80,04/08/2019 03:24:00 +0000,8.90,7.60,5.96,8.90,8.74,0,0.00,0.62,...,68,7.490000,9.200000,11.370000,13.480000,15.206667,191.703333,28.215000,43.483333,987.810000
4,8.40,04/08/2019 03:24:30 +0000,9.72,7.72,6.14,9.72,9.66,153937,1.03,531.46,...,68,7.490000,9.200000,11.360000,13.466667,15.193333,191.676667,28.205000,43.381667,987.810000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,6.08,04/10/2019 10:52:30 +0000,8.64,5.62,4.94,8.72,8.18,73639,0.49,255.12,...,52,5.913333,6.883333,7.923333,12.833333,20.376667,204.200000,29.251667,30.115000,980.160000
2059,6.86,04/10/2019 10:53:00 +0000,9.72,5.98,5.20,33.96,11.90,0,0.00,0.62,...,52,5.923333,6.853333,7.813333,12.610000,20.093333,205.363333,29.228333,30.173333,980.280000
2060,7.72,04/10/2019 10:54:00 +0000,11.14,6.82,5.10,30.24,13.58,0,0.00,0.62,...,52,5.986667,6.866667,7.680000,12.440000,19.880000,208.336667,29.238333,30.260000,980.038333
2061,7.46,04/10/2019 10:54:30 +0000,10.02,6.66,5.12,33.36,14.00,0,0.00,0.62,...,52,6.023333,6.903333,7.716667,12.476667,19.916667,209.700000,29.180000,30.353333,980.218333


In [7]:
dict_col_names = {'pm1Palas':'PM$_{1.0}$ in $\mu g/m^3 $ (Palas)', 
                  'pm2_5Palas': 'PM$_{2.5}$ in $\mu g/m^3$ (Palas)',
                  'pm4Palas': 'PM$_{4.0}$ in $\mu g/m^3$ (Palas)',
                  'pm10Palas': 'PM$_{10.0}$ in $\mu g/m^3$ (Palas)',
                  'pmTotalPalas': 'Total PM Concentration in $\mu g/m^3$ (Palas)', 
                  'dCnPalas': 'Particle Count Density in #/cm$^{3}$ (Palas)', 
                  'P1_lpo_loRa': '> 1 μm LPO (LoRa)',
                  'P1_ratio_loRa': '> 1 μm Ratio (LoRa)',
                  'P1_conc_loRa':'> 1 μm Concentration in $\mu g/m^3$ (LoRa)' , 
                  'P2_lpo_loRa': '> 2.5 μm LPO (LoRa)',
                  'P2_ratio_loRa': '> 2.5 μm Ratio (LoRa)', 
                  'P2_conc_loRa': '> 2.5 μm Concentration in $\mu g/m^3$ (LoRa)',
                  'Temperature_loRa': 'Temperature in ℃ (LoRa)', 
                  'Pressure_loRa': 'Pressure in Pa (LoRa)',
                  'Humidity_loRa': 'Humidity in % (LoRa)'} 

In [8]:
filtered_data = df.iloc[:,7:-3]
filtered_data

,P1_lpo_loRa,P1_ratio_loRa,P1_conc_loRa,P2_lpo_loRa,P2_ratio_loRa,P2_conc_loRa,Temperature_loRa,Pressure_loRa,Humidity_loRa,pm1Palas,pm2_5Palas,pm4Palas,pm10Palas,pmTotalPalas,dCnPalas
0,2556,0.02,9.48,151843,1.01,524.26,20.53,98866,67,7.500000,9.210000,11.390000,13.516667,15.256667,191.830000
1,0,0.00,0.62,67639,0.45,234.43,20.38,98866,68,7.493333,9.210000,11.386667,13.506667,15.246667,191.786667
2,86608,0.58,299.81,332245,2.21,1145.71,20.31,98869,68,7.490000,9.206667,11.380000,13.493333,15.226667,191.736667
3,0,0.00,0.62,30289,0.20,105.48,20.27,98868,68,7.490000,9.200000,11.370000,13.480000,15.206667,191.703333
4,153937,1.03,531.46,285021,1.90,982.52,20.20,98868,68,7.490000,9.200000,11.360000,13.466667,15.193333,191.676667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,73639,0.49,255.12,162046,1.08,559.33,20.69,98121,52,5.913333,6.883333,7.923333,12.833333,20.376667,204.200000
2059,0,0.00,0.62,7320,0.05,25.99,20.69,98125,52,5.923333,6.853333,7.813333,12.610000,20.093333,205.363333
2060,0,0.00,0.62,0,0.00,0.62,20.70,98123,52,5.986667,6.866667,7.680000,12.440000,19.880000,208.336667
2061,0,0.00,0.62,15928,0.11,55.80,20.69,98122,52,6.023333,6.903333,7.716667,12.476667,19.916667,209.700000


In [9]:
col_name = list(filtered_data)
x =[]
y_Palas  = []
for i in col_name:
    if "_loRa" in i:
        x.append(i)
    if "Palas" in i:
        y_Palas.append(i)
Palas = {}
for i in y_Palas:
    Palas_cols = x + [i]
    Palas[i[:-len("Palas")]] = filtered_data[Palas_cols]
y_Palas

['pm1Palas', 'pm2_5Palas', 'pm4Palas', 'pm10Palas', 'pmTotalPalas', 'dCnPalas']

In [10]:
def plot_residuals(predict_test,y_test,filtered_data):
    idx = X_test.index
    data_test = filtered_data.loc[idx]
    data_test["dateTime"] = pd.to_datetime(data_test["dateTime"])
    residuals = (np.array(y_test) - predict_test)**2
    plt.figure()
    plt.scatter(data_test["dateTime"], residuals)
    plt.xticks(rotation = 45)
    plt.xlabel('Date Time')
    plt.ylabel(y_test.name)
    plt.show()

In [11]:
dict_col_regression = {'pm1Palas':'PM$_{1.0}$', 
                       'pm2_5Palas': 'PM$_{2.5}$',
                       'pm4Palas': 'PM$_{4.0}$',
                       'pm10Palas': 'PM$_{10.0}$',
                       'pmTotalPalas': 'Total PM Concentration', 
                       'dCnPalas': 'Particle Count Density',
                       'dateTime': 'Date Time',
                       'P1_lpo_loRa':'> 1 μm LPO',
                       'P1_ratio_loRa': '> 1 μm ratio', 
                       'P1_conc_loRa': '> 1 μm Concentration', 
                       'P2_lpo_loRa': '> 2.5 μm LPO', 
                       'P2_ratio_loRa': '> 2.5 μm ratio',
                       'P2_conc_loRa': '> 2.5 μm Concentration' , 
                       'Temperature_loRa': 'Temperature', 
                       'Pressure_loRa': 'Pressure', 
                       'Humidity_loRa':'Humidity'}
unit_regression = {'pm_conc':'($\mu g/m^3$)',
                   'dCn':'(#/cm$^{3}$)'} 

In [12]:
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/models/Linear_Regression.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/models/Decision_Tree_Regression.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/models/Ensemble_Bagging_Regression.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/models/LGBM_Regression.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/models/Ridge_Regression.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/models/KNN_Regression.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/models/Neural_Network_Regression.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/models/XGBoost_Regression.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/models/Random_Forest_Regression.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/models/Stacking_Regression.ipynb

In [13]:
r2_score_test_lr ={}
r2_score_test_rr = {}
r2_score_test_knn = {}
r2_score_test_dt ={}
r2_score_test_xgb = {}
r2_score_test_lgbm ={}
r2_score_test_br ={}
r2_score_test_nn ={}
r2_score_test_rf ={}
# r2_score_test_sl = {}

for k,v in enumerate(Palas):
    X = Palas[v].drop([v+"Palas"],axis = 1)
    y = Palas[v][v+"Palas"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    X_train.to_csv("X_train.csv")
    X_test.to_csv("X_test.csv")
    y_train.to_csv("y_train.csv")
    y_test.to_csv("y_test.csv")

    r2_score_test_lr[v] =  Linear_Regression(X_train,X_test,y_train,y_test,filtered_data)
    r2_score_test_rr[v] = Ridge_Regression(X_train,X_test,y_train,y_test,filtered_data)
    r2_score_test_knn[v] = KNN_Regression(X_train,X_test,y_train,y_test,filtered_data)
    r2_score_test_dt[v] = Decision_Tree_Regression(X_train,X_test,y_train,y_test,filtered_data)   
    r2_score_test_xgb[v] = XGBoost_Regression(X_train,X_test,y_train,y_test,filtered_data)
    r2_score_test_lgbm[v]  =  LGBM_Regression(X_train,X_test,y_train,y_test,filtered_data)
    r2_score_test_br[v] = Ensemble_Bagging_Regression(X_train,X_test,y_train,y_test,filtered_data)
    r2_score_test_nn[v] = Neural_Network_Regression(X_train,X_test,y_train,y_test,filtered_data)
    r2_score_test_rf[v] = Random_Forest_Regression(X_train,X_test,y_train,y_test,filtered_data)
    
   

R2 value of train data 0.48
R2 value of test data 0.49
        Actual  Predicted  Category
0     7.500000   7.097484  Training
1     7.493333   7.117532  Training
2     7.490000   7.188907  Training
3     7.490000   7.072395  Training
4     7.490000   7.089140  Training
...        ...        ...       ...
2058  5.913333   8.088949  Training
2059  5.923333   8.055103  Training
2060  5.986667   8.057350  Training
2061  6.023333   8.071296  Training
2062  6.053333   8.072036  Training

[2063 rows x 3 columns]
r2 train 0.48
r2 test 0.49
r2 train 0.95
r2 test 0.93
R2 value of train data 1.0
R2 value of test data 0.97
        Actual  Predicted  Category
0     7.500000   7.484854  Training
1     7.493333   7.488678  Training
2     7.490000   7.484769  Training
3     7.490000   7.470900  Training
4     7.490000   7.487444  Training
...        ...        ...       ...
2058  5.913333   5.913859  Training
2059  5.923333   5.936957  Training
2060  5.986667   5.960980  Training
2061  6.023333   6.0

In [14]:
import pandas as pd

models = [
    "Linear_Regression",
    "Random_Forest_Regression",
    "XGBoost_Regression",
    "Decision_Tree_Regression",
    "LGBM_Regression",
    "Ensemble_Bagging_Regression",
    "Neural_Network_Regression",
    "Ridge_Regression",
    "KNN_Regression"
]

v_values = ["pm1", "pm2_5", "pm4", "pm10", "pmTotal", "dCn"]

# Initialize a DataFrame to store the R^2 scores
data = {
    "Model": models
}

for v in v_values:
    data[v] = [
        r2_score_test_lr[v],
        r2_score_test_rf[v],
        r2_score_test_xgb[v],
        r2_score_test_dt[v],
        r2_score_test_lgbm[v],
        r2_score_test_br[v],
        r2_score_test_nn[v],
        r2_score_test_rr[v],
        r2_score_test_knn[v]
    ]

# Create the DataFrame
r2_score_table = pd.DataFrame(data)

# Sort the table by decreasing order of R^2 scores for each column in v_values
for v in v_values:
    r2_score_table = r2_score_table.sort_values(by=v, ascending=False)

# Save the sorted table to a CSV file
r2_score_table.to_csv("sorted_r2_score_table.csv", index=False)

# Display the sorted table
print(r2_score_table)


                         Model   pm1  pm2_5   pm4  pm10  pmTotal   dCn
5  Ensemble_Bagging_Regression  0.99   0.99  0.99  0.91     0.83  0.99
1     Random_Forest_Regression  0.99   0.99  0.99  0.91     0.82  0.99
4              LGBM_Regression  0.98   0.98  0.99  0.90     0.81  0.99
3     Decision_Tree_Regression  0.97   0.98  0.99  0.83     0.62  0.99
2           XGBoost_Regression  0.98   0.99  0.98  0.87     0.81  0.98
6    Neural_Network_Regression  0.95   0.94  0.87  0.63     0.43  0.95
8               KNN_Regression  0.93   0.94  0.93  0.53     0.44  0.94
7             Ridge_Regression  0.49   0.49  0.54  0.12     0.20  0.40
0            Linear_Regression  0.49   0.49  0.54  0.12     0.20  0.39
